In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

# -*- coding: utf-8 -*-
"""chexpert_noisfy.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Up66s8R2xK4yRqBX0sGYIZpyZ1ncOnhd
"""

#torch imports
import torch
torch.set_printoptions(profile="full")
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

#general imports
import os
import json
import pickle
import shutil
import datetime
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

x = '/content/gdrive/My\ Drive/chexpert_semantic_noise/noisy_data'
%cd $x
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/chexpert_semantic_noise/noisy_data
baseline		      train_acc        val_acc
buffer			      train_f1_score   val_f1_score
models			      train_loss       val_loss
network_training_predictions  train_precision  val_precision
__pycache__		      train_real_acc   val_real_acc
tensorboard_logs	      train_real_loss  val_real_loss
torch_chexpert_dataset.py     train_recall     val_recall


# INITIALIZE

In [2]:
import os
import sys
import h5py
import time
import json
import pickle
import datetime
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
#torch imports
import torch
torch.set_printoptions(profile="full")
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
# some metrics
from sklearn.metrics import recall_score, precision_score, f1_score
# some local files
sys.path.append('baseline')
from tf_chexpert_utilities import *
from torch_chexpert_dataset import ChexpertDataset
# handle ssl problem
#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context

def fpath(folder, noise):
    return '%s/n_%s'%(folder, str(noise))

"""## model compile func"""
def compile_model(model, binary=False):
  # Instantiate a logistic loss function that expects integer targets.
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  # Instantiate an accuracy metric.
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  if binary:
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.binary_accuracy
  
  # Instantiate an optimizer.
  optimizer = tf.keras.optimizers.Adam()
  # Instantiate some callbacks
  
  model.compile(optimizer=optimizer, loss=loss,metrics=[accuracy])
  return model

def initialize_logger(fname, train_length, val_length, test_length):
    if os.path.exists(fname):
      # if log file exists remove it permanately
      return h5py.File(fname, 'a')

    logger = h5py.File(fname, 'a')
    logger.create_dataset('train_loss', (train_length, ), compression='gzip')
    logger.create_dataset('train_real_loss', (train_length, ), compression='gzip')
    logger.create_dataset('train_acc', (train_length, ), compression='gzip')
    logger.create_dataset('train_real_acc', (train_length, ), compression='gzip')
    logger.create_dataset('train_precision', (train_length, ), compression='gzip')
    logger.create_dataset('train_recall', (train_length, ), compression='gzip')
    logger.create_dataset('train_f1_score', (train_length, ), compression='gzip')

    logger.create_dataset('val_loss', (train_length, ), compression='gzip')
    logger.create_dataset('val_real_loss', (train_length, ), compression='gzip')
    logger.create_dataset('val_acc', (train_length, ), compression='gzip')
    logger.create_dataset('val_real_acc', (train_length, ), compression='gzip')
    logger.create_dataset('val_precision', (train_length, ), compression='gzip')
    logger.create_dataset('val_recall', (train_length, ), compression='gzip')
    logger.create_dataset('val_f1_score', (train_length, ), compression='gzip')
    return logger


def log_metrics(logger_file, KEY, EPOCH, loss, real_loss, acc, real_acc, precision, recall, _f1Score):
    save_ndarray(f'{prediction_save_folder}/{KEY}_loss_{EPOCH}', loss)
    save_ndarray(f'{prediction_save_folder}/{KEY}_real_loss_{EPOCH}', real_loss)
    save_ndarray(f'{prediction_save_folder}/{KEY}_acc_{EPOCH}', acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_real_acc_{EPOCH}', real_acc)
    save_ndarray(f'{prediction_save_folder}/{KEY}_precision_{EPOCH}', precision)
    save_ndarray(f'{prediction_save_folder}/{KEY}_recall_{EPOCH}', recall)
    save_ndarray(f'{prediction_save_folder}/{KEY}_f1_score_{EPOCH}', _f1Score)

    #logger_file[f'{KEY}_loss'][EPOCH]       = loss
    #logger_file[f'{KEY}_real_loss'][EPOCH]  = real_loss
    #logger_file[f'{KEY}_acc'][EPOCH]        = acc
    #logger_file[f'{KEY}_real_acc'][EPOCH]   = real_acc
    #logger_file[f'{KEY}_precision'][EPOCH]  = precision
    #logger_file[f'{KEY}_recall'][EPOCH]     = recall
    #logger_file[f'{KEY}_f1_score'][EPOCH]   = _f1Score

"""## GET SAVED HDF5 DATASET FILE"""    
# get h5py DATASET file
dset = h5py.File('./buffer/baseline/dset.hdf5', 'r')
train_dataset = dset['train']
val_dataset = dset['val']
test_dataset = dset['test']

"""## SHUFFLING INDICES"""
# SHUFFLE TRAIN INDICES
np.random.seed(1234)
train_shuffled_idx_lst = np.arange(len(train_dataset))
np.random.shuffle(train_shuffled_idx_lst)
print("shuffled_train_indices: %s"%str(train_shuffled_idx_lst))
# SHUFFLE VAL INDICES
np.random.seed(5678)
val_shuffled_idx_lst = np.arange(len(val_dataset))
np.random.shuffle(val_shuffled_idx_lst)
print("shuffled_val_indices: %s"%str(val_shuffled_idx_lst))
# SHUFFLE TEST INDICES
np.random.seed(9012)
test_shuffled_idx_lst = np.arange(len(test_dataset))
np.random.shuffle(test_shuffled_idx_lst)
print("shuffled_test_indices: %s"%str(test_shuffled_idx_lst))
print('-'*30)

"""## GETTING THE GROUND TRUTH AND NOISY LABELS"""
train_noisy_label_dict  = {}
val_noisy_label_dict    = {}
train_ground_truth      = get_noisy_labels(0.0, 'train', len(train_dataset))
val_ground_truth        = get_noisy_labels(0.0, 'val', len(val_dataset))
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  - - - - - - - - 
noises = [0, 0.1, 0.2, 0.3, 0.4, 0.5, .60]
for n in noises:
  train_noisy_label_dict[n] = get_noisy_labels(n, 'train', len(train_dataset))
  val_noisy_label_dict[n]   = get_noisy_labels(n, 'val', len(val_dataset))


"""## SOME TRAINING PARAMATERS"""
# get custom datasets for network training

# GET SHUFFLED NAMES/LABES FOR TRAIN
train_ground_truth  = train_ground_truth[train_shuffled_idx_lst]
val_ground_truth    = val_ground_truth[val_shuffled_idx_lst]
for n in noises:
    train_noisy_label_dict[n]   =  train_noisy_label_dict[n][train_shuffled_idx_lst]
    val_noisy_label_dict[n]     =  val_noisy_label_dict[n][val_shuffled_idx_lst]

tensorboard_log_dir = './tensorboard_logs'
network_training_pred_folder = './network_training_predictions'
prediction_save_folder = './network_training_predictions/torch_baseline'
model_dir = './models'
model_save_dir = './models/torch_baseline'

make_sure_folder_exists(tensorboard_log_dir)
make_sure_folder_exists(network_training_pred_folder)
make_sure_folder_exists(prediction_save_folder)
make_sure_folder_exists(model_dir)
make_sure_folder_exists(model_save_dir)
make_sure_folder_exists('%s/%s'%(prediction_save_folder, '0'))
for n in noises:
    make_sure_folder_exists('%s/%i'%(prediction_save_folder, int(n*100)))



shuffled_train_indices: [ 2280  6728  5664 ... 23605  1318 25299]
shuffled_val_indices: [2018 2622 2091 ... 2366 2199 2164]
shuffled_test_indices: [2739 2604  179 ... 3302 2522 3314]
------------------------------
Given folder: ./tensorboard_logs EXISTS.
Given folder: ./network_training_predictions EXISTS.
Given folder: ./network_training_predictions/torch_baseline EXISTS.
Given folder: ./models EXISTS.
Given folder: ./models/torch_baseline EXISTS.
Given folder: ./network_training_predictions/torch_baseline/0 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/0 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/10 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/20 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/30 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/40 EXISTS.
Given folder: ./network_training_predictions/torch_baseline/50 EXISTS.
Given folder: ./network_training_predictions/torch_b

# SET ARGS

In [3]:
# get args
# # # # if you are using a notebook comment the code below and uncomment the ' NOETBOOK MODE ' code block# # # #
#opt = get_args()

#BATCH_SIZE = opt.batch_size
#MAX_EPOCHS = opt.max_epoch
#NOISE_RATIO = opt.noise_ratio

#LR = opt.lr
#MOMENTUM = opt.momentum
#WEIGHT_DECAY = opt.weight_decay
#RESUME = opt.resume
skip_mean = False
# # # # NOTEBOOK MODE # # # #
BATCH_SIZE = 16
MAX_EPOCHS = 100
NOISE_RATIO = 0.2
LR = 0.02
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
RESUME = 1
#skip_mean = True

# open h5py file to save the epoch states
#logger = initialize_logger('%s/%i/0_35_log.hdf5'%(prediction_save_folder, int(100*NOISE_RATIO)), len(train_dataset), len(val_dataset), len(test_dataset))

In [4]:
MAX_EPOCHS

100

# COMPUTE MEAN AND MAKE LOADERS READY

In [5]:
np_train_dataset = np.array(train_dataset, dtype=np.uint8)[train_shuffled_idx_lst]

In [6]:
np_val_dataset = np.array(val_dataset, dtype=np.uint8)[val_shuffled_idx_lst]

In [7]:
np_test_dataset = np.array(test_dataset, dtype=np.uint8)[test_shuffled_idx_lst]

In [8]:
if not skip_mean:
  # Get mean to initialize transform
  trainset = ChexpertDataset(r=NOISE_RATIO, hdf5_dataset=np_train_dataset, noisy_labels=train_noisy_label_dict[NOISE_RATIO], batch_size=BATCH_SIZE, ground_truth=train_ground_truth, transform=transforms.ToTensor())
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
  mean = 0
  before = datetime.datetime.now()    
  for i, data in enumerate(trainloader, 0):
      imgs, labels, real_labels, index= data
      mean += torch.from_numpy(np.mean(np.asarray(imgs), axis=(2,3))).sum(0)
      if (i+1)%15 == 0:
          print_remaining_time(before, i+1, len(trainloader))
  mean = mean / len(trainset)
else:
  mean = [0.5, 0.5, 0.5]
# get transform and initialize trainset and trainloader
transform_train = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomCrop(256, padding=4), 
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.ToTensor(),
                                      transforms.Normalize((mean[0], mean[1], mean[2]),(1.0, 1.0, 1.0))])
                                      
transform_test = transforms.Compose([transforms.Resize(256),
                                     transforms.ToTensor(),
                                     transforms.Normalize((mean[0], mean[1], mean[2]), (1.0, 1.0, 1.0))])


15/1678(0.89%) finished. Estimated Remaining Time: 0.00 seconds.
30/1678(1.79%) finished. Estimated Remaining Time: 0.00 seconds.
45/1678(2.68%) finished. Estimated Remaining Time: 0.00 seconds.
60/1678(3.58%) finished. Estimated Remaining Time: 26.97 seconds.
75/1678(4.47%) finished. Estimated Remaining Time: 21.37 seconds.
90/1678(5.36%) finished. Estimated Remaining Time: 17.64 seconds.
105/1678(6.26%) finished. Estimated Remaining Time: 14.98 seconds.
120/1678(7.15%) finished. Estimated Remaining Time: 12.98 seconds.
135/1678(8.05%) finished. Estimated Remaining Time: 22.86 seconds.
150/1678(8.94%) finished. Estimated Remaining Time: 20.37 seconds.
165/1678(9.83%) finished. Estimated Remaining Time: 18.34 seconds.
180/1678(10.73%) finished. Estimated Remaining Time: 16.64 seconds.
195/1678(11.62%) finished. Estimated Remaining Time: 15.21 seconds.
210/1678(12.51%) finished. Estimated Remaining Time: 20.97 seconds.
225/1678(13.41%) finished. Estimated Remaining Time: 19.37 seconds.


In [9]:
print(f'mean: {mean}')

mean: tensor([0.4385, 0.4385, 0.4385])


In [10]:
trainset = ChexpertDataset(r=NOISE_RATIO, hdf5_dataset=np_train_dataset, noisy_labels=train_noisy_label_dict[NOISE_RATIO], batch_size=BATCH_SIZE, ground_truth=train_ground_truth, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

valset = ChexpertDataset(r=NOISE_RATIO, hdf5_dataset=np_val_dataset, noisy_labels=val_noisy_label_dict[NOISE_RATIO], batch_size=BATCH_SIZE, ground_truth=val_ground_truth, transform=transform_test)
valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f'train loader length: {len(trainset)}')
print(f'val loader length: {len(valset)}')

train loader length: 26848
val loader length: 3356


# LOAD AND TRAIN

In [11]:
# get densenet
net = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=False)

# change the last classifier to predict 2 class instead of 10
num_ftrs = net.classifier.in_features
net.classifier = nn.Linear(in_features=num_ftrs, out_features=2, bias=True)
# get criterion
criterion = nn.CrossEntropyLoss()

net.cuda()
criterion.cuda()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

num_classes = 2
train_preds = np.zeros((len(trainset), num_classes)) - 1.
val_preds = np.zeros((len(valset), num_classes)) - 1.

if RESUME == 1:
  fn = os.path.join(model_save_dir, 'checkpoint.pth.tar')
  ckpt = torch.load(fn)
  epoch_resume = ckpt['epoch']
  best_val_acc = ckpt['best_val_acc']
  net.load_state_dict(ckpt['state_dict'])
  optimizer.load_state_dict(ckpt['optimizer'])
  print('loading network SUCCESSFUL')
else:
  epoch_resume = 0
  best_val_acc = 0
  print('Starting training WITHOUT loading a network...') 

# TRAINING
for epoch in range(epoch_resume, MAX_EPOCHS):
  before = datetime.datetime.now()
  train_loss = 0
  train_acc = 0
  train_real_loss = 0
  train_real_acc = 0
  recall = 0
  precision = 0
  _f1Score = 0
  for i, data in enumerate(trainloader, 0):
    net.zero_grad()
    imgs, labels, real_ground_truths, index = data 
        # imgs               : data
        # labels             : noisy labels
        # real ground truths : not-noisy ground truths
        # index              : sample index
    imgs = Variable(imgs.cuda().float())
    labels = Variable(labels.cuda().long()[:,1])
    real_ground_truths = Variable(real_ground_truths.cuda().long())
        # forward
    logits = net(imgs.float())
        # get the accuracy and predictions of the current batch
    _, pred = torch.max(logits.data, -1)
    acc = float((pred==labels.data).sum()) 
    train_acc += acc
    train_real_acc += float((pred==real_ground_truths.data[:,1]).sum()) 
        # get current loss of the batch
    current_loss = criterion(logits, labels)
    train_loss += imgs.size(0) * current_loss.data    
    train_real_loss += imgs.size(0) * criterion(logits, real_ground_truths[:,1]).data
        # backward
    current_loss.backward()
    optimizer.step()
        # put prediction history in a list to save it later
    softmax_pred        = F.softmax(logits, -1).cpu().data
    train_preds[index.cpu()] = softmax_pred
        # put the metrics that COMPARES THE REAL GROUND TRUTHS
    recall    += imgs.size(0) * recall_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,    average="samples")
    precision += imgs.size(0) * precision_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5, average="samples")
    _f1Score  += imgs.size(0) * f1_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,        average="samples")
    if (i+1)%(len(trainloader)//4)==0:
      print_remaining_time(before, i+1, len(trainloader))
        
  # get avarage loss and accuracy
  train_loss  /= len(trainset)
  train_acc   /= len(trainset)
  recall      /= len(trainset)
  precision   /= len(trainset)
  _f1Score    /= len(trainset)
  train_real_loss /= len(trainset)
  train_real_acc  /= len(trainset)
  
  print('TRAIN EPOCH END [%6d/%6d] loss: %5f, real_loss: %5f, acc: %5f, real_acc:%5f, recall: %5f, precision: %5f, f1_score: %5f' %(epoch, MAX_EPOCHS, train_loss, train_real_loss, train_acc, train_real_acc, recall, precision, _f1Score))

  # HANLE LOGS
  log_metrics(
    None, 
    KEY='train', 
    EPOCH=epoch, 
    loss=train_loss.cpu(),
    real_loss=train_real_loss.cpu(),
    acc=train_acc,
    real_acc=train_real_acc,
    recall=recall,
    precision=precision,
    _f1Score = _f1Score)
  
    # reset the placeholder for the predictions in the next epoch 
  train_preds = train_preds*0 - 1.
    # VALIDATION STATE
  net.eval()
  val_loss = 0.0
  val_real_loss = 0.0
  val_acc = 0.0
  val_real_acc = 0.0
  val_prec = 0
  val_f1_score = 0
  val_recall = 0
  before = datetime.datetime.now()
  with torch.no_grad():
    for i, data in enumerate(valloader, 0):
      imgs, labels, real_ground_truths, index = data
      imgs = Variable(imgs.cuda())
      labels = Variable(labels.cuda().long()[:,1])
      real_ground_truths = Variable(real_ground_truths.cuda().long())
            # LOSS and REAL LOSS
      logits = net(imgs.float())
      loss = criterion(logits, labels)
      val_loss += imgs.size(0)*loss.data
      val_real_loss += imgs.size(0) * criterion(logits, real_ground_truths[:,1]).data
            # ACC and REAL ACC
      _, pred = torch.max(logits.data, -1)
      acc = float((pred==labels.data).sum())
      val_acc += acc
      val_real_acc += float((pred==real_ground_truths.data[:,1]).sum()) 
            # put prediction history in a list to save it later
      softmax_pred            = F.softmax(logits, -1).cpu().data
      val_preds[index.cpu()]  = softmax_pred
            # put the metrics that COMPARES THE REAL GROUND TRUTHS
      val_recall    += imgs.size(0) * recall_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,    average="samples")
      val_prec      += imgs.size(0) * precision_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5, average="samples")
      val_f1_score  += imgs.size(0) * f1_score(real_ground_truths.cpu(), softmax_pred.cpu() > 0.5,        average="samples")
      if (i+1)%(len(valloader)//4) == 0:
        print_remaining_time(before, i+1, len(valloader))

  val_loss      /= len(valset)
  val_acc       /= len(valset)
  val_recall    /= len(valset)
  val_prec      /= len(valset)
  val_f1_score  /= len(valset)
  val_real_loss /= len(valset)
  val_real_acc  /= len(valset)
    ####################################################
    # log the validation
  is_best = val_acc > best_val_acc
  best_val_acc = max(val_acc, best_val_acc)

  log_metrics(
    None, 
    KEY='val', 
    EPOCH=epoch, 
    loss=val_loss.cpu(),
    real_loss=val_real_loss.cpu(),
    acc=val_acc,
    real_acc=val_real_acc,
    recall=val_recall,
    precision=val_prec,
    _f1Score=val_f1_score)
  ####################################################
  print('\tVALIDATION...loss: %5f, acc: %5f, best_acc: %5f, real_acc: %5f, real_loss: %5f, recall: %5f, precision: %5f f1_score: %5f'%(val_loss, val_acc, best_val_acc, val_real_acc, val_real_loss, val_recall, val_prec, val_f1_score))
    ####################################################
  net.train()
    ####################################################
  val_preds = val_preds * 0 - 1.
  state = ({'epoch' : epoch,'state_dict' : net.state_dict(),'optimizer' : optimizer.state_dict(),'best_val_acc' : best_val_acc})

  fn = os.path.join(model_save_dir, 'checkpoint.pth.tar')
  best_fn = os.path.join(model_save_dir, 'best_val_checkpoint.pth.tar')
  print(f'Saving model to: {fn}')
  torch.save(state, fn)
  # save state of the best validation    
  if is_best:
    print(f'Saving best val acc model to: {best_fn}')
    torch.save(state, best_fn)
  after = datetime.datetime.now()
  print(f'total time passed for the epoch {epoch}: {(after-before).seconds} seconds...')

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip


loading network SUCCESSFUL
419/1678(24.97%) finished. Estimated Remaining Time: 192.31 seconds.
838/1678(49.94%) finished. Estimated Remaining Time: 128.31 seconds.
1257/1678(74.91%) finished. Estimated Remaining Time: 64.31 seconds.
1676/1678(99.88%) finished. Estimated Remaining Time: 0.31 seconds.
TRAIN EPOCH END [    42/   100] loss: 0.610051, real_loss: 0.808380, acc: 0.648801, real_acc:0.577622, recall: 0.577622, precision: 0.577622, f1_score: 0.577622


/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


52/210(24.76%) finished. Estimated Remaining Time: 9.12 seconds.
104/210(49.52%) finished. Estimated Remaining Time: 5.10 seconds.
156/210(74.29%) finished. Estimated Remaining Time: 2.77 seconds.
208/210(99.05%) finished. Estimated Remaining Time: 0.11 seconds.
	VALIDATION...loss: 0.665604, acc: 0.601013, best_acc: 0.620083, real_acc: 0.549464, real_loss: 0.813592, recall: 0.549464, precision: 0.549464 f1_score: 0.549464
Saving model to: ./models/torch_baseline/checkpoint.pth.tar
total time passed for the epoch 42: 13 seconds...
419/1678(24.97%) finished. Estimated Remaining Time: 189.30 seconds.
838/1678(49.94%) finished. Estimated Remaining Time: 125.30 seconds.
1257/1678(74.91%) finished. Estimated Remaining Time: 62.97 seconds.
1676/1678(99.88%) finished. Estimated Remaining Time: 0.30 seconds.
TRAIN EPOCH END [    43/   100] loss: 0.608160, real_loss: 0.806233, acc: 0.649769, real_acc:0.584699, recall: 0.584699, precision: 0.584699, f1_score: 0.584699
52/210(24.76%) finished. Est

update 46

# pytorch playground

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

ValueError: ignored

In [ ]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_classes  =10
train_preds  	  = torch.zeros(len(trainset), num_classes) - 1.
num_hist = 10
train_preds_hist  = torch.zeros(len(trainset), num_hist, num_classes)
pl_ratio = 0.; nl_ratio = 1.-pl_ratio
train_losses      = torch.zeros(len(trainset)) - 1.
CC = 300
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    train_acc = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels, index = data
        #index = torch.ones(32) + i*32
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # get the predictions
        _, pred = torch.max(outputs.data, -1)      
        acc = float((pred==labels.data).sum())
        train_acc += acc
        # # # - # # # - # # #
        train_preds[index.cpu()] = F.softmax(outputs, -1).cpu().data
        train_losses[i] = loss.data
        # # # - # # # - # # #
        loss.backward()
        optimizer.step()
        # # # - # # # - # # #
        # print statistics
        running_loss += loss.item()
        if i % CC == CC - 1:    # print every CC mini-batches)
            print('[%d, %5d] loss: %.3f acc: %.3f' % (epoch + 1, i + 1, running_loss / CC, train_acc/(CC)))
            running_loss = 0.0
            train_acc = 0.0

print('Finished Training')

ValueError: ignored